# Creating a translation function for flir and clustering

In [ ]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.speedups

In [ ]:
def move_flir_to_rgb(flir_identifications):

    flir_translation_constant = (1.0272989925397269e-07*3, 1.5250405240863074e-06)
    flir_identifications = flir_identifications.reset_index()
    for index, row in flir_identifications.iterrows():
        flir_identifications.loc[index, 'lat'] = flir_identifications.loc[index,'lat'] + flir_translation_constant[1]
        flir_identifications.loc[index, 'lon'] = flir_identifications.loc[index, 'lon'] - flir_translation_constant[0]
        
    return flir_identifications

def overlay(rgb, flir, buffersize = .000001):
    flir = move_flir_to_rgb(flir)
    flir_gdf = gpd.GeoDataFrame(flir, geometry=gpd.points_from_xy(flir.lon, flir.lat))
    rgb = gpd.GeoDataFrame(rgb, geometry=gpd.points_from_xy(rgb.lon, rgb.lat))
    rgb['geometry'] = rgb.geometry.buffer(buffersize)
    flir_gdf['geometry'] = flir_gdf.geometry.buffer(buffersize)  
    nearby_points = gpd.overlay( rgb, flir_gdf, how='intersection')
    nearby_points.plot(alpha=0.5, figsize = (20,40))
    
    return nearby_points


def name_flir(rgb, flir, buffersize = .000001):
    flir = move_flir_to_rgb(flir)
    flir_gdf = gpd.GeoDataFrame(flir, geometry=gpd.points_from_xy(flir.lon, flir.lat))
    rgb = gpd.GeoDataFrame(rgb, geometry=gpd.points_from_xy(rgb.lon, rgb.lat))
#     rgb['geometry'] = rgb.geometry.buffer(buffersize)
    flir_gdf['geometry'] = flir_gdf.geometry.buffer(buffersize)  
    
    
    shapely.speedups.enable() 
    # enable shapely.speedups which makes queries running faster.
    rgb = rgb.reset_index()
    flir_gdf = flir_gdf.reset_index()
    
    flir_gdf['plant_name'] = 'NaN'
    
    for index, row in flir_gdf.iterrows():
        
#         print(index)
    
        
        pip_mask = rgb.within(row['geometry'])
        
#         print(pip_mask)
        
        rgb_points = rgb.loc[pip_mask]
        
#         print(rgb_points)
        
        if len(rgb_points) > 1:
#             rgb_points = rgb_points.iloc[rgb_points['bounding_area_m2'].argmax()]
            rgb_points = rgb_points[rgb_points['genotype'] == row['genotype']]
            if len(rgb_points) > 1:
                rgb_points = rgb_points[rgb_points['bounding_area_m2'] == max(rgb_points['bounding_area_m2'])]
        
#         print('rgb',rgb_points.iloc[0]['genotype'])
        
        try:
#             print('flir',flir_gdf.loc[index]['genotype'], 'rgb', rgb_points.iloc[0]['genotype'])
            if str(flir_gdf.loc[index]['genotype']) ==  str(rgb_points.iloc[0]['genotype']):
#                 print('worked')
                flir_gdf.loc[[index],['plant_name']] =  rgb_points.iloc[0]['plant_name']
#                 print(flir_gdf.loc[index])
#                 print(flir.gdf.loc[index]['plant_name'])
#             print(rgb_points['plant_name'])
            
        except:
            print('')
#     Backwards
#     pip_mask = flir_gdf.within(rgb.loc[0, 'geometry']) 
#     # selecting the polygon's geometry field to filter out points that 
#     # are not overlaid
#     flir_point = flir_gdf.loc[pip_mask]
#     name= rgb.loc[0,'plant_name']
    
    return flir_gdf

In [ ]:
flir_identifications =  pd.read_csv(r"C:\Users\Travis\AppData\Local\Packages\CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc\LocalState\rootfs\home\travis_s\flir_identifications\2020-03-03_indv_temps.csv")

# flir_identifications = pd.read_csv(r"C:\Users\Travis\Downloads\FLIR_indv_plant_temp_S10\2020-03-03_indv_temps.csv")

rgb_consistent_identifications = pd.read_csv(r"C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\every_day_identifications.csv")

In [ ]:
rgb_last_day = rgb_consistent_identifications[rgb_consistent_identifications['date'] == '2020-03-03']



In [ ]:
named_and_moved_flir = name_flir(rgb_last_day, flir_identifications, buffersize = .000004)



In [ ]:
named_and_moved_flir.head()

named_plants = named_and_moved_flir[named_and_moved_flir['plant_name'] != 'NaN']

In [ ]:
named_plants.to_csv(r'C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\moved_named_flir_test.csv')

In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
named_and_moved_flir.head()

## Named and moved looks great 2/14/2021, now use this to name all of the other days of flir and add together.

In [ ]:
import glob

In [ ]:
last_day_named_flir = pd.read_csv(r"C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\moved_named_flir.csv")

flir_identifications_list =  glob.glob(r"C:\Users\Travis\AppData\Local\Packages\CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc\LocalState\rootfs\home\travis_s\flir_identifications\*.csv")


In [ ]:
all_flir = pd.DataFrame()

for i in flir_identifications_list[:-1]:
    df = pd.read_csv(i)
    all_flir = all_flir.append(df)

all_flir.head()
    

In [ ]:
all_flir.head(-5)

In [ ]:
# testing if this just works like this

all_days_flir = name_flir(last_day_named_flir, all_flir, buffersize = .000004)
all_days_flir.head()

In [ ]:
all_days_flir.to_csv(r'C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\all_days_flir_moved_named.csv')

In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [6]:
# combining all days

In [1]:
import pandas as pd

In [5]:
most_days = pd.read_csv(r"C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\moved_named_flir.csv")
last_day = pd.read_csv(r"C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\all_days_flir_moved_named.csv")


all_days = most_days.append(last_day)

# all_days.reset_index()

all_days.to_csv(r"C:\Users\Travis\OneDrive - University of Arizona\2_9_2021 Lettuce Data\season_10_flir_identifications_moved_named_complete.csv")